In [1]:
import os
import sys
import random
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
from tensorflow.keras import layers, Model, Sequential


seed_num = 7
random.seed(seed_num)
np.random.seed(seed_num)


from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from evaluation import stratified_concordance_index, stratified_brier_score
import requests


In [2]:
class BaseFeedForwardNet(tf.keras.Model):
    """Implements the Neural Network Feed Forward Base Class to be used with different Loss Functions."""

    def __init__(self, input_dim, output_dim, hidden_dims=[500, 200, 100, 10]):
        """
        Initializes network architecture.

        Args:
            input_dim (int): Dimensionality of input vector
            output_dim (int): Dimensionality of output vector
        """
        super(BaseFeedForwardNet, self).__init__()
        self.layers_list = []

        current_dim = input_dim
        for hdim in hidden_dims:
            self.layers_list.append(tf.keras.layers.Dense(hdim, activation='relu'))
            self.layers_list.append(tf.keras.layers.Dropout(0.4))
            current_dim = hdim

        self.layers_list.append(tf.keras.layers.Dense(output_dim))

    def call(self, x):
        output = x
        for layer in self.layers_list:
            output = layer(output)
        return output


class StratifiedPartialLikelihoodLoss(tf.keras.losses.Loss):
    """Implements the Stratified Partial Likelihood loss function as a custom TensorFlow loss function."""

    def __init__(self):
        super(StratifiedPartialLikelihoodLoss, self).__init__()

    def partial_likelihood(self, output, event_time, event_indicator):
        """
        Calculates the negative stratified partial likelihood on neural network output.
        """
        sorted_ind = tf.argsort(event_time)
        output = tf.gather(output, sorted_ind)
        event_indicator = tf.gather(event_indicator, sorted_ind)
        output_uncensored = tf.boolean_mask(output, event_indicator)

        accumulated_risk = tf.math.log(
            tf.reverse(
                tf.math.cumsum(tf.reverse(tf.math.exp(output), axis=[0])), axis=[0]
            )
        )

        uncensored_accumulated_risk = tf.boolean_mask(accumulated_risk, event_indicator)
        return -tf.reduce_sum(output_uncensored - uncensored_accumulated_risk)

    def call(self, y_true, y_pred):
        output, event_time, event_indicator, strata = y_pred, y_true[:, 0], y_true[:, 1], y_true[:, 2]

        if strata is None:
            strata = tf.fill(event_indicator.shape, 1)

        unique_groups, _ = tf.unique(strata)
        p_losses = []

        for strat in unique_groups:
            indices_strata = tf.where(strata == strat)[:, 0]
            p_losses.append(self.partial_likelihood(
                tf.gather(output, indices_strata),
                tf.gather(event_time, indices_strata),
                tf.gather(event_indicator, indices_strata),
            ))

        return tf.reduce_sum(p_losses)




def create_keras_feedforward_model(input_dim, output_dim, hidden_dims=[500, 200, 100, 10]):
    """
    Creates a Keras Sequential model equivalent to the BaseFeedForwardNet.

    Args:
        input_dim (int): Dimensionality of input vector
        output_dim (int): Dimensionality of output vector
        hidden_dims (list): List of hidden layer dimensions

    Returns:
        tf.keras.Model: Keras model equivalent to BaseFeedForwardNet
    """
    model = Sequential()
    
    # Input layer
    model.add(layers.Input(shape=(input_dim,)))

    # Hidden layers
    for hdim in hidden_dims:
        model.add(layers.Dense(hdim, activation='relu'))

    # Output layer
    model.add(layers.Dense(output_dim))

    return model




## Load sample dataset

In [3]:
def load_npy_from_url(url):
    """
    Downloads a .npy file from a given URL and loads it as a NumPy array.

    Parameters:
        url (str): The URL of the .npy file (must be a Raw URL for GitHub files).

    Returns:
        np.ndarray: The loaded NumPy array.
    """
    try:
        # Download the file from the URL
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)

        # Save the content to a temporary file
        with open('temp.npy', 'wb') as temp_file:
            temp_file.write(response.content)

        # Load the NumPy array from the temporary file
        data = np.load('temp.npy', allow_pickle=True)
        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching the file: {e}")
        return None
    except Exception as e:
        print(f"Error loading the .npy file: {e}")
        return None

In [4]:
url_X_train = 'https://raw.githubusercontent.com/JunetaeKim/GCSP-HBDA/main/Week11/Data/X_train.npy'
url_X_val = 'https://raw.githubusercontent.com/JunetaeKim/GCSP-HBDA/main/Week11/Data/X_val.npy'
url_y_train = 'https://raw.githubusercontent.com/JunetaeKim/GCSP-HBDA/main/Week11/Data/y_train.npy'
url_y_val = 'https://raw.githubusercontent.com/JunetaeKim/GCSP-HBDA/main/Week11/Data/y_val.npy'

X_train = load_npy_from_url(url_X_train)
X_val = load_npy_from_url(url_X_val)
y_train = load_npy_from_url(url_y_train)
y_val = load_npy_from_url(url_y_val)

# This is an arbitrary method to facilitate the research setup and is not a standard approach. Please keep this in mind.
y_train[:, 0]= y_train[:, 0] * 200
y_val[:, 0]= y_val[:, 0] * 200

# GitHub Raw URL
response = requests.get('https://raw.githubusercontent.com/JunetaeKim/GCSP-HBDA/main/Week11/Data/x_varlist.pkl')
x_varlist = pickle.loads(response.content)


X_train_DF = pd.DataFrame(X_train, columns=x_varlist)
X_val_DF = pd.DataFrame(X_val, columns=x_varlist)
Y_train_DF = pd.DataFrame(y_train, columns=['time','event', 'stratum'])
Y_val_DF = pd.DataFrame(y_val, columns=['time','event', 'stratum'])




### Parameter setting

In [5]:
input_dim = X_train.shape[1]
output_dim = 1
hidden_dims = [500, 200, 100, 50]
batch_size = 500
epochs = 200
weight_save_path = './Save'

### Model Training

In [6]:
# Define model, loss and optimizer
model = BaseFeedForwardNet(input_dim, output_dim, hidden_dims)
loss_fn = StratifiedPartialLikelihoodLoss()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

best_score = np.inf
num_batches = int(np.ceil(len(X_train) / batch_size))  
for epoch in range(epochs):
    # Shuffle the data at the start of each epoch
    permutation = np.random.permutation(len(X_train))
    X_train = X_train[permutation]
    y_train = y_train[permutation]
    
    for i in range(num_batches):
        X_batch = X_train[i * batch_size: (i + 1) * batch_size]
        y_batch = y_train[i * batch_size: (i + 1) * batch_size]

        with tf.GradientTape() as tape:
            y_pred = model(X_batch, training=True)
            loss = loss_fn(y_batch, y_pred)

        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    val_loss = loss_fn(y_val, model(X_val, training=False))

    if val_loss <= best_score:
        best_score = val_loss
        model.save_weights(weight_save_path + '/StratfiedNN.h5')  # Save the model weights
        print('Model saved at epoch of ', epoch)
        

    # Print loss for each epoch
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}, val_loss: {val_loss.numpy()}")

Model saved at epoch of  0
Epoch 1, Loss: 79.87882995605469, val_loss: 956.329345703125
Model saved at epoch of  1
Epoch 2, Loss: 65.66600036621094, val_loss: 954.9307250976562


KeyboardInterrupt: 

### Load trained weights from the local disk

In [7]:
model.load_weights(weight_save_path + '/StratfiedNN.h5')
keras_model = create_keras_feedforward_model(input_dim, output_dim, hidden_dims)
keras_model.set_weights([w.numpy() for w in model.trainable_weights])


In [8]:
BRIER_EVAL_TIME = int(min(np.max(y_train[:, 0]), np.max(y_val[:, 0]))) # End of the investigated time period for which the brier score is computed 


# Structured arrays for Brier Score Evaluation.
survival_data_train = np.zeros(y_train.shape[0],
    dtype={'names':('event_indicator', 'event_time'), 'formats':('bool', 'u2')})
survival_data_train['event_indicator'] = y_train[:, 1]
survival_data_train['event_time'] = y_train[:, 0]

survival_data_test = np.zeros(y_val.shape[0],
    dtype={'names':('event_indicator', 'event_time'), 'formats':('bool', 'u2')})
survival_data_test['event_indicator'] = y_val[:, 1]
survival_data_test['event_time'] = y_val[:, 0]

strata_train = y_train[:, 2]
strata_test = y_val[:, 2]


numpy_output_train = tf.squeeze(model(X_train), axis=1).numpy()
numpy_output_test = tf.squeeze(model(X_val), axis=1).numpy()


brier_score = stratified_brier_score(
    BRIER_EVAL_TIME,
    survival_data_train,
    survival_data_test,
    numpy_output_train,
    numpy_output_test,
    strata_train=strata_train,
    strata_test=strata_test
)


c_index = stratified_concordance_index(
    numpy_output_test,
    survival_data_test['event_indicator'],
    survival_data_test['event_time'],
    strata_test
)